In [ ]:
import numpy as np
import tensorflow as tf

# Load and preprocess the text
with open('/content/Semed_Vurgun_seirleri.txt', 'r', encoding='utf-8') as f:
    text = f.read().lower()

In [ ]:
text[:100]

'\ncavanlara xi̇tab\nmənim düşkün könlüm sizdən eləyir\nbu fərz*\n olan təmənnanı, cavanlar.\nunutmayın bə'

In [ ]:
''.join(sorted(set(text.lower())))

'\x01\n !"\'()*,-./0123456789:;<>?[\\]^abcdefghijklmnopqrstuvwxyz§°»äçöüğışə̇–—‘’“”†‡…★'

In [ ]:
import re
text = re.sub(r'[^a-zA-Zəğıöçşüı\s\n]', '', text)

In [ ]:
''.join(sorted(set(text.lower())))

'\n abcdefghijklmnopqrstuvwxyzçöüğışə'

In [ ]:
text[:100]

'\ncavanlara xitab\nmənim düşkün könlüm sizdən eləyir\nbu fərz\n olan təmənnanı cavanlar\nunutmayın bəşər '

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split = "character",
                                                   standardize = 'lower')

text_vec_layer.adapt(text)
encoded = text_vec_layer([text][0])

In [ ]:
encoded -= 2
vocab_size = text_vec_layer.vocabulary_size() -2
dataset_size = len(encoded)

In [ ]:
vocab_size

35

In [ ]:
dataset_size

794641

In [ ]:
def to_dataset(sequence, length, seed =None, shuffle = False, batch_size = 32):

    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window: window.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.batch(batch_size)

    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).repeat().prefetch(1)

In [ ]:
length  =100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:600_000], length = length, shuffle = True,
                       seed = 42)
valid_set = to_dataset(encoded[600_000:675_000], length = length)
test_set = to_dataset(encoded[675_000:], length = length)

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = 16),
    tf.keras.layers.LSTM(128, return_sequences = True),
    tf.keras.layers.Dense(vocab_size, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Nadam(learning_rate = 1e-3),
              metrics = ['accuracy'])

model_ckpt = tf.keras.callbacks.ModelCheckpoint('Semed_Vurgun.keras',
                                                monitor = "val_accuracy",
                                                save_best_only = True)

early_stop = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy",
                                              patience = 3,
                                              restore_best_weights = True)
num_windows = 600_000 - length
steps_per_epoch = num_windows // 32

val_windows = 75_000 - length
validation_steps = val_windows // 32

model.fit(
    train_set,
    validation_data = valid_set,
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    epochs = 10,
    callbacks = [model_ckpt, early_stop]
)

Epoch 1/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 159s 8ms/step - accuracy: 0.4768 - loss: 1.7340 - val_accuracy: 0.4199 - val_loss: 2.0565
Epoch 2/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 155s 8ms/step - accuracy: 0.5749 - loss: 1.4139 - val_accuracy: 0.4356 - val_loss: 1.9843
Epoch 3/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 156s 8ms/step - accuracy: 0.5773 - loss: 1.3950 - val_accuracy: 0.4453 - val_loss: 1.9340
Epoch 4/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 156s 8ms/step - accuracy: 0.5758 - loss: 1.3887 - val_accuracy: 0.4486 - val_loss: 1.9221
Epoch 5/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 157s 8ms/step - accuracy: 0.5774 - loss: 1.3819 - val_accuracy: 0.4512 - val_loss: 1.8995
Epoch 6/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 157s 8ms/step - accuracy: 0.5765 - loss: 1.3807 - val_accuracy: 0.4560 - val_loss: 1.8821
Epoch 7/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 157s 8ms/step - accuracy: 0.5764 - loss: 1.3782 - val_accuracy: 0.4564 - val_loss: 1.8726
Epoch 8/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 157s 8ms/step - ac

In [ ]:
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda x: x-2), #No <PAD> or <UNK> tokens
    model
])

In [ ]:
y_proba = model.predict(tf.constant(["Sala"]))[0,-1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred+2]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


np.str_('r')

In [ ]:
def next_char(text, temprature):
    text = tf.constant([text])
    y_proba = model.predict(text)[0,-1:]
    rescaled_logits = tf.math.log(y_proba) / temprature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1)[0,0].numpy()
    return text_vec_layer.get_vocabulary()[char_id+2 ]

In [ ]:
def extend_text(text, chars = 20, temprature = 1):
    for _ in range(chars):
        text += next_char(text, temprature)
    return text

In [ ]:
extend_text('Azərbaycan', chars = 30, temprature = 0.2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

'Azərbaycan\nbir də bu gün kimi bir haqqı '

In [ ]:
vocab_size = vocab_size
max_length = 100
embed_size = 16
num_heads = 2
ff_dim = 64

inputs = tf.keras.Input(shape=(None,), dtype='int32')
embed_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size)
embedding = embed_layer(inputs)

pos_embed_layer = tf.keras.layers.Embedding(input_dim=max_length, output_dim=embed_size)

positions = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit=tf.shape(x)[1], delta=1))(inputs)
pos_embed_dec = pos_embed_layer(positions)

embed = embedding + pos_embed_dec

causal_mask = tf.keras.layers.Lambda(
    lambda x: tf.linalg.band_part(tf.ones((tf.shape(x)[1], tf.shape(x)[1])), -1, 0)
)(inputs)

decoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, \
                                                       key_dim=embed_size)(embed, embed, attention_mask=causal_mask)
decoder_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(embed + decoder_attention)

decoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(decoder_attention)
decoder_ff = tf.keras.layers.Dense(embed_size)(decoder_ff)
decoder_outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_attention + decoder_ff)

decoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, \
                                                       key_dim=embed_size)(decoder_outputs, decoder_outputs, attention_mask=causal_mask)
decoder_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_outputs + decoder_attention)

decoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(decoder_attention)
decoder_ff = tf.keras.layers.Dense(embed_size)(decoder_ff)
decoder_outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_attention + decoder_ff)

outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder_outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy']
)

model.fit(
    train_set,
    validation_data=valid_set,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=10,
    callbacks=[model_ckpt, early_stop]
)

Epoch 1/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 103s 5ms/step - accuracy: 0.3186 - loss: 2.2079 - val_accuracy: 0.3797 - val_loss: 2.0248
Epoch 2/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 94s 5ms/step - accuracy: 0.3982 - loss: 1.9313 - val_accuracy: 0.3932 - val_loss: 1.9845
Epoch 3/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 90s 5ms/step - accuracy: 0.4148 - loss: 1.8765 - val_accuracy: 0.3974 - val_loss: 1.9720
Epoch 4/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 90s 5ms/step - accuracy: 0.4244 - loss: 1.8458 - val_accuracy: 0.4012 - val_loss: 1.9480
Epoch 5/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 89s 5ms/step - accuracy: 0.4303 - loss: 1.8268 - val_accuracy: 0.4056 - val_loss: 1.9482
Epoch 6/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 90s 5ms/step - accuracy: 0.4327 - loss: 1.8164 - val_accuracy: 0.4051 - val_loss: 1.9393
Epoch 7/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 90s 5ms/step - accuracy: 0.4343 - loss: 1.8085 - val_accuracy: 0.4089 - val_loss: 1.9255
Epoch 8/10
18746/18746 ━━━━━━━━━━━━━━━━━━━━ 89s 5ms/step - accuracy:

In [ ]:
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda x: x-2), #No <PAD> or <UNK> tokens
    model
])

In [ ]:
y_proba = model.predict(tf.constant(["Sala"]))[0,-1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred+2]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


np.str_('r')

In [ ]:
def next_char(text, temprature):
    text = tf.constant([text])
    y_proba = model.predict(text)[0,-1:]
    rescaled_logits = tf.math.log(y_proba) / temprature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1)[0,0].numpy()
    return text_vec_layer.get_vocabulary()[char_id+2 ]

In [ ]:
def extend_text(text, chars = 20, temprature = 1):
    for _ in range(chars):
        text += next_char(text, temprature)
    return text

In [ ]:
extend_text('Azərbaycan', chars = 30, temprature = 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━

'Azərbaycanın bir yeri\ngəlir bu dalğaları'